In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("compas-scores-two-years-violent.csv")
df = df[
    (df['days_b_screening_arrest'] <= 30) &
    (df['days_b_screening_arrest'] >= -30) &
    (df['is_recid'] != -1) &
    (df['c_charge_degree'] != "O") &
    (df['score_text'] != 'N/A')
]
# 사전재판(pretrial)” 대상만 남기는 필터링은 다른 변수 조합으로 간접적으로 구현

In [ ]:
print(df.columns)


Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid', 'two_year_recid.1'],
      dtype='object')


In [ ]:
# 동일 날짜에 여러 COMPAS 점수가 있을 경우 → ID 큰 것 채택
df = df.sort_values(['id'], ascending=True)
df = df.drop_duplicates(subset=['id', 'screening_date'], keep='last')




In [ ]:
#목표: “현재 범죄(current offense)”를 정의하고 누락된 경우 제거

# 날짜 형식으로 변환
df['screening_date'] = pd.to_datetime(df['screening_date'])
df['c_offense_date'] = pd.to_datetime(df['c_offense_date'], errors='coerce')

# screening 이전/당일 사건만 남기고, 그중 가장 최근 사건 선택
def get_current_offense(group):
    # screening 날짜
    screening_date = group['screening_date'].iloc[0]

    # 1차: screening 이전 or 당일의 사건 중 가장 최근
    past_cases = group[group['c_offense_date'] <= screening_date]

    if not past_cases.empty:
        latest_case = past_cases.loc[past_cases['c_offense_date'].idxmax()]
        return latest_case

    # 2차: screening 후 30일 이내 사건 중 가장 가까운 것
    within_30_days = group[
        (group['c_offense_date'] > screening_date) &
        (group['c_offense_date'] <= screening_date + pd.Timedelta(days=30))
    ]

    if not within_30_days.empty:
        nearest_case = within_30_days.loc[within_30_days['c_offense_date'].idxmin()]
        return nearest_case

    # 3차: 둘 다 해당 안 되면 None 반환 → 나중에 제거
    return None

# 각 사람 + screening 날짜마다 하나의 사건만 남기기
df_current = df.groupby(['id', 'screening_date'], group_keys=False).apply(get_current_offense)

# None (현재 범죄 못 찾은 경우) 제거
df_current = df_current.dropna(subset=['c_offense_date'])

<ipython-input-47-8cdb2bb4b947>:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_current = df.groupby(['id', 'screening_date'], group_keys=False).apply(get_current_offense)


In [ ]:
# 경범죄 제거 (charge_degree == 'O'인 경우 제외)
df = df[df['c_charge_degree'] != 'O']


In [ ]:
# 폭력적 범죄 판별 규칙 설정 (여기서는 예시로 'assault', 'battery' 등을 포함한 경우 폭력적이라고 판단)
violent_keywords = ['assault', 'battery', 'robbery', 'murder', 'shoot', 'gun', 'weapon']

# 폭력성 판단 컬럼 생성
df['is_violent'] = df['c_charge_desc'].apply(lambda x: any(keyword in str(x).lower() for keyword in violent_keywords))

# 폭력적 범죄가 아닌 경우는 False로 채워짐
df['is_violent'] = df['is_violent'].fillna(False)


In [ ]:
# 현재 범죄 이후 데이터 제외
df_current = df_current[df_current['c_offense_date'] <= df_current['screening_date']]

df_current.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid', 'two_year_recid.1'],
      dtype='object')

In [ ]:
# df_current와 기존 df 병합
df_current = df_current.reset_index(drop=True)
df = pd.merge(df, df_current[['id', 'screening_date', 'c_offense_date', 'c_charge_desc', 'is_violent_recid']],
              on=['id', 'screening_date'], how='left')

# 서브스케일 계산
def calculate_subscale(row):
    if row['score_text'] == 'Low':
        return 1
    elif row['score_text'] == 'Medium':
        return 2
    elif row['score_text'] == 'High':
        return 3
    else:
        return np.nan  # 결측값 처리

df['subscale'] = df.apply(calculate_subscale, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, r2_score

df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date_x', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc_x', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid_x', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid', 'two_year_recid.1',
       'is_violent', 'c_offense_date_y', 'c_ch

In [ ]:
# 특성 변수 (Features) 설정
features = [
    'age', 'priors_count', 'days_b_screening_arrest', 'c_charge_degree',
    'decile_score', 'is_violent'  # or 'is_violent_recid_x' or 'is_violent_recid_y' 등
]

# 타겟 변수 (Target) 설정
target = 'two_year_recid'  # 타겟 변수는 'two_year_recid'

In [ ]:
# 결측치가 있는 행 제거
df_clean = df.dropna(subset=features + [target])

# 특성과 타겟 변수 분리
X = df_clean[features]
y = df_clean[target]

# 훈련 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [ ]:
from sklearn.preprocessing import LabelEncoder
# c_charge_degree 컬럼을 숫자형으로 변환
label_encoder = LabelEncoder()
df_clean['c_charge_degree'] = label_encoder.fit_transform(df_clean['c_charge_degree'])


In [ ]:
# 특성과 타겟 변수 분리
X = df_clean[features]
y = df_clean[target]


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
# 훈련 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 학습 (XGBoost)
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# 모델 학습 (RandomForest)
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [ ]:
# XGBoost 모델 예측
y_pred_xgb = xgb_model.predict(X_test)

# XGBoost 모델 평가 (RMSE와 R2)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

# 셋째자리까지 반올림 후 출력
print("XGBoost RMSE: {:.3f}".format(rmse_xgb))
print("XGBoost R2: {:.3f}".format(r2_xgb))

# RandomForest 모델 예측
y_pred_rf = rf_model.predict(X_test)

# RandomForest 모델 평가 (RMSE와 R2)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

# 셋째자리까지 반올림 후 출력
print("Random Forest RMSE: {:.3f}".format(rmse_rf))
print("Random Forest R2: {:.3f}".format(r2_rf))


XGBoost RMSE: 0.410
XGBoost R2: -0.232
Random Forest RMSE: 0.413
Random Forest R2: -0.250


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 선형 회귀 모델 학습
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# 예측
y_pred_lr = lr_model.predict(X_test)

# RMSE 계산 (수정된 부분)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

# R^2 계산
r2_lr = r2_score(y_test, y_pred_lr)

# 결과 출력
print(f"Linear Regression - RMSE: {rmse_lr:.3f}, R2: {r2_lr:.3f}")


Linear Regression - RMSE: 0.342, R2: 0.145


In [ ]:
from sklearn.svm import SVR

# 서포트 벡터 머신 모델 학습
svm_model = SVR()
svm_model.fit(X_train, y_train)

# 예측
y_pred_svm = svm_model.predict(X_test)

# RMSE 계산
rmse_svm = np.sqrt(mean_squared_error(y_test, y_pred_svm))

# R^2 계산
r2_svm = r2_score(y_test, y_pred_svm)

# 결과 출력
print(f"SVM - RMSE: {rmse_svm:.3f}, R2: {r2_svm:.3f}")


SVM - RMSE: 0.364, R2: 0.030
